In [16]:
import re
import os
import nltk
import tensorflow as tf
import numpy as np
import pandas as pd
from nltk.corpus import reuters
import collections
from transformers import pipeline

%load_ext nb_black

<IPython.core.display.Javascript object>

In [19]:
documents = reuters.fileids()
train_docs = list(filter(lambda doc: doc.startswith("train"), documents))
test_docs = list(filter(lambda doc: doc.startswith("test"), documents))
categories = reuters.categories()

<IPython.core.display.Javascript object>

In [20]:
title_re = "([A-Z].*[a-z]$)"


def title_split(doc):
    prev_word = ""
    for word in doc:
        if bool(re.match(title_re, word)):
            title_pos = doc.index(word)
            if prev_word == "A" or word == "USA":
                title_pos = title_pos - 1
            return " ".join(doc[:title_pos])
        prev_word = word


def words_split(doc):
    prev_word = ""
    for word in doc:
        if bool(re.match(title_re, word)):
            title_pos = doc.index(word)
            if prev_word == "A" or word == "USA":
                title_pos = title_pos - 1
            return " ".join(doc[title_pos:])
        prev_word = word

<IPython.core.display.Javascript object>

In [21]:
train_list = [int(x.split("/")[1]) for x in train_docs]
cat_list = [reuters.categories(x) for x in train_docs]
title_list = [title_split(reuters.words(x)) for x in train_docs]
words_list = [words_split(reuters.words(x)) for x in train_docs]

<IPython.core.display.Javascript object>

In [22]:
news_df = pd.DataFrame(
    {
        "doc_id": train_list,
        "categories": cat_list,
        "title": title_list,
        "content": words_list,
    }
)

<IPython.core.display.Javascript object>

In [23]:
def title_split(row):
    split_word = row["content"].split()[0]
    title_new = split_word + row["content"].split(split_word)[1]
    row["title"] = title_new
    row["content"] = title_new
    return row

<IPython.core.display.Javascript object>

In [24]:
title_split_df = news_df[news_df["title"] == ""].apply(title_split, axis=1)
cols = list(news_df.columns)
news_df.loc[news_df.doc_id.isin(title_split_df.doc_id), cols] = title_split_df[cols]

<IPython.core.display.Javascript object>

In [26]:
cat_dict = {
    "acq": "acorn capital investment fund limited",
    "alum": "aluminum",
    "bop": "bottom of the pyramid",
    "castor-oil": "castor oil",
    "carcass": "livestock",
    "cpu": "computershare",
    "coconut-oil": "coconut oil",
    "copra-cake": "copra cake",
    "cpi": "consumer price index",
    "dfl": "deutsche fussball liga",
    "dlr": "digital realty trust",
    "dmk": "dravida munnetra kazhagam",
    "earn": "ellington residential mortgage reit stock",
    "gnp": "grupo nacional provincial",
    "groundnut-oil": "ground nut oil",
    "instal-debt": "instalco intressenter debt",
    "ipi": "intrepid potash",
    "iron-steel": "iron steel",
    "l-cattle": "cattle",
    "lei": "leading economic index",
    "lin-oil": "linseed oil",
    "meal-feed": "meal feed",
    "money-fx": "money foreign exchange",
    "money-supply": "money supply",
    "nat-gas": "natural gas",
    "nkr": "nokia",
    "palm-oil": "palm oil",
    "palmkernel": "palm kernel",
    "pet-chem": "petronas chemicals",
    "rape-oil": "rapeseed oil",
    "soy-meal": "soy meal",
    "soy-oil": "soy oil",
    "strategic-metal": "strategic metal",
    "sun-meal": "sunflower meal",
    "sun-oil": "sunflower oil",
    "sunseed": "sunflower seed",
    "veg-oil": "vagetable oil",
    "wpi": "waterfront philippines",
}

<IPython.core.display.Javascript object>

In [27]:
news_df["top_label"] = news_df["categories"].apply(lambda x: x[0])
news_df["title_length"] = news_df["title"].apply(lambda x: len(x) if x != None else 0)
news_df["con_length"] = news_df["content"].apply(lambda x: len(x) if x != None else 0)
news_df["top_label"] = news_df["top_label"].apply(
    lambda x: cat_dict[x] if x in cat_dict.keys() else x
)

<IPython.core.display.Javascript object>

In [41]:
news_df.to_pickle("C:/Users/rparg/Documents/Data/Reuters/news_df.pkl")

<IPython.core.display.Javascript object>

In [28]:
new_cats = [
    "Acorn Capital Investment Fund Limited",
    "aluminum",
    "bottom of the pyramid",
    "barley",
    "castor oil",
    "carcass",
    "cattle",
    "cocoa",
    "coconut",
    "coconut oil",
    "coffee",
    "consumer price index",
    "copper",
    "copra cake",
    "corn",
    "cotton",
    "computershare",
    "crude",
    "deutsche fussball liga",
    "digital realty trust",
    "dravida munnetra kazhagam",
    "ellington residential mortgage reit stock",
    "fuel",
    "gas",
    "gold",
    "grain",
    "groundnut",
    "ground nut oil",
    "grupo nacional provincial",
    "heat",
    "hog",
    "housing",
    "income",
    "instalco intressenter debt",
    "interest",
    "intrepid potash",
    "iron steel",
    "jet",
    "jobs",
    "lead",
    "leading economic index",
    "linseed oil",
    "livestock",
    "lumber",
    "meal feed",
    "money foreign exchange",
    "money supply",
    "natural gas",
    "naphtha",
    "nickel",
    "nokia",
    "new zealand dollar",
    "oat",
    "orange",
    "palladium",
    "palm oil",
    "palm kernel",
    "petronas chemicals",
    "platinum",
    "potato",
    "propane",
    "rand",
    "rapeseed",
    "rapeseed oil",
    "reserves",
    "retail",
    "rice",
    "rubber",
    "rye",
    "ship",
    "silver",
    "sorghum",
    "soybean",
    "soy meal",
    "soy oil",
    "strategic metal",
    "sugar",
    "sunflower meal",
    "sunflower oil",
    "sunflower seed",
    "tea",
    "tin",
    "trade",
    "vagetable oil",
    "waterfront philippines",
    "wheat",
    "yen",
    "zinc",
]

<IPython.core.display.Javascript object>

In [32]:
news_df = pd.read_pickle("C:/Users/rparg/Documents/Data/Reuters/news_df.pkl")

<IPython.core.display.Javascript object>

## Load Zero-Shot Classifier

In [33]:
classifier = pipeline("zero-shot-classification", model="roberta-large-mnli")

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at roberta-large-mnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


<IPython.core.display.Javascript object>

## Zero-Shot Classification of 16 top-categories with 30+ docs

In [34]:
cat_count_df = news_df.groupby(by="top_label").count()
cat_count_df.sort_values(by="doc_id", ascending=False).iloc[:35]

,doc_id,categories,title,content,title_length,con_length
top_label,,,,,,
ellington residential mortgage reit stock,2843,2843,2693,2693,2843,2843
acuisition,1650,1650,1494,1494,1650,1650
crude,370,370,331,331,370,370
interest,329,329,285,285,329,329
money foreign exchange,266,266,238,238,266,266
trade,253,253,230,230,253,253
grain,218,218,204,204,218,218
corn,157,157,137,137,157,157
digital realty trust,126,126,109,109,126,126


<IPython.core.display.Javascript object>

In [36]:
sub_cat_list = [
    "aluminum",
    "barley",
    "cocoa",
    "coffee",
    "copper",
    "corn",
    "cotton",
    "crude",
    "gold",
    "grain",
    "iron steel",
    "livestock",
    "natural gas",
    "palm oil" "rubber",
    "sugar",
]
sub_cat_df = news_df[news_df["top_label"].isin(sub_cat_list)]

<IPython.core.display.Javascript object>

In [43]:
sub_cat_df = sub_cat_df[sub_cat_df["con_length"] > 500]
sc_sample_df = (
    sub_cat_df.groupby("top_label")
    .apply(lambda x: x.sample(20, replace=True))
    .reset_index(drop=True)
)
sc_sample_df = sc_sample_df.drop(
    columns=["categories", "title_length", "con_length"]
).drop_duplicates()
sc_sample_df.to_pickle("C:/Users/rparg/Documents/Data/Reuters/sc_sample.pkl")

<IPython.core.display.Javascript object>

In [44]:
sc_sample_list = list(sc_sample_df["content"])

<IPython.core.display.Javascript object>

In [ ]:
sc_class_list = [
    classifier(content, sub_cat_list, multi_label=True) for content in sc_sample_list
]
categorized_df = pd.DataFrame.from_records(sc_class_list)
categorized_df.to_pickle("C:/Users/rparg/Documents/Data/Reuters/sc_cat.pkl")

## Zero shot classification based on document titles

In [12]:
news_filt_df = news_df[news_df["title_length"] > 40]
sample_df = (
    news_filt_df.groupby("top_label")
    .apply(lambda x: x.sample(15, replace=True))
    .reset_index(drop=True)
)
sample_df = sample_df.drop(columns=["categories"]).drop_duplicates()
sample_df.to_pickle("C:/Users/rparg/Documents/Data/Reuters/sample_titles.pkl")

<IPython.core.display.Javascript object>

In [13]:
class_title_list = list(sample_df["title"])

<IPython.core.display.Javascript object>

In [14]:
clfd_dict_list2 = [
    classifier(title, new_cats, multi_label=True) for title in class_title_list
]
categorized_df = pd.DataFrame.from_records(clfd_dict_list2)
categorized_df.to_pickle("C:/Users/rparg/Documents/Data/Reuters/categorized2.pkl")

<IPython.core.display.Javascript object>

## Zero shot classification based on document titles

In [13]:
news_con_filt_df = news_df[
    (news_df["con_length"] > 200) & (news_df["con_length"] < 1000)
]

<IPython.core.display.Javascript object>

In [14]:
sample_con_df = (
    news_con_filt_df.groupby("top_label")
    .apply(lambda x: x.sample(4, replace=True))
    .reset_index(drop=True)
)
sample_con_df = sample_con_df.drop(columns=["categories"]).drop_duplicates()
sample_con_df.to_pickle("C:/Users/rparg/Documents/Data/Reuters/sample_content.pkl")

<IPython.core.display.Javascript object>

In [15]:
class_con_list = list(sample_con_df["content"])

<IPython.core.display.Javascript object>

In [16]:
clfd_dict_list2 = [
    classifier(title, new_cats, multi_label=True) for title in class_con_list
]
categorized_df = pd.DataFrame.from_records(clfd_dict_list2)
categorized_df.to_pickle("C:/Users/rparg/Documents/Data/Reuters/con_cat.pkl")

<IPython.core.display.Javascript object>